In [ ]:
#Import the SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

In [ ]:
#Create the variable to hold the VADER lexicon object
obj = SentimentIntensityAnalyzer()

PRACTICE

In [ ]:
#Create list with review
trail_reviews = ['Dirty. This place has been ruined by people with no respect for nature. Diapers and femine hygiene products left everywhere.Super crowded. Skip and go elsewhere.']

In [ ]:
#Print the score followed by the review
#Use for loop if there are multiple reviews in the list
for review in trail_reviews:
    scores = obj.polarity_scores(review)
    print(scores['compound'],':',review)

TRAIL REVIEWS

In [ ]:
#Load reviews dataset UPDATE WITH FILE PATH AND NAME
trail_reviews_df = pd.read_csv('Resources/TripAdvisor_Reviews.csv')
trail_reviews_df

In [ ]:
#Remove aposthrophes from Reviews
trail_reviews_df["Review"].str.replace("'","")

## Reviews by Trail

In [ ]:
#Group reviews in one row per trail
reviews_by_trail_df = trail_reviews_df.groupby('Trail Name')['Review'].apply(''.join).reset_index()

reviews_by_trail_df

In [ ]:
#Generate Sentiment scores
#Assume the review is stored in "Review" column
reviews_by_trail_df['scores']=reviews_by_trail_df['Review'].apply(lambda trail_overview: obj.polarity_scores(str(trail_overview)))

In [ ]:
#Create column for compound score
reviews_by_trail_df['compound'] = reviews_by_trail_df['scores'].apply(lambda score_dict: score_dict['compound'])

#Create column for RF Model (1 = overall postive, 0 = overall negative)
reviews_by_trail_df['RF_Sentiment']=''
reviews_by_trail_df.loc[reviews_by_trail_df.compound>=0,'RF_Sentiment']=1
reviews_by_trail_df.loc[reviews_by_trail_df.compound<0,'RF_Sentiment']=0

reviews_by_trail_df

In [ ]:
reviews_by_trail_df["RF_Sentiment"].value_counts()

## Score each review and calculate mean compound score for each trail

In [ ]:
#Generate Sentiment scores
trail_reviews_df['scores']=trail_reviews_df['Review'].apply(lambda trail_overview: obj.polarity_scores(str(trail_overview)))

In [ ]:
#Create column for compound score
trail_reviews_df['compound'] = trail_reviews_df['scores'].apply(lambda score_dict: score_dict['compound'])

#Create column for RF Model (1 = overall postive, 0 = overall negative)
trail_reviews_df['RF_Sentiment']=''
trail_reviews_df.loc[trail_reviews_df.compound>0.75,'RF_Sentiment']=1
trail_reviews_df.loc[trail_reviews_df.compound<=0.75,'RF_Sentiment']=0

trail_reviews_df

In [ ]:
trail_reviews_df["RF_Sentiment"].value_counts()

In [ ]:
#Group trail rows and calculate the trail's average compound score
review_summary = pd.DataFrame(trail_reviews_df.groupby('Trail Name')["compound",].mean())

#Create column for RF Model (1 = overall postive, 0 = overall negative)
review_summary['RF_Sentiment']=''
review_summary.loc[review_summary.compound>0.75,'RF_Sentiment']=1
review_summary.loc[review_summary.compound<=0.75,'RF_Sentiment']=0


review_summary

In [ ]:
review_summary.to_csv(r"Resources/TripAdvisor_Reviews_VADER.csv", index=True)